In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, MinMaxScaler
from sklearn.metrics import mean_absolute_error
#import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
#import pyodbc
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.kernel_approximation import RBFSampler
import sys
#import funkcije as f
from IPython.display import display, Markdown
#import seaborn as sns
from sklearn.impute import SimpleImputer
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
gradiskatrain = pd.read_csv('../podaci/df_gradiska_train_inter.csv')
gradiskatest = pd.read_csv('../podaci/df_gradiska_test_inter.csv')


#gradiskatrain = gradiskatrain.dropna()
#gradiskatest = gradiskatest.dropna()

print(gradiskatrain.head(10))


preprocess = Pipeline([
 
    ('poly', PolynomialFeatures(degree=2)),
    ('min_max', MinMaxScaler())
])


training_columns = ['AvgPixelCount','NumberOfDaysFromStart', 'PreviousWeight']


X_train = gradiskatrain[training_columns]
y_train = gradiskatrain['AverageWeight']
X_test = gradiskatest[training_columns]
y_test = gradiskatest['AverageWeight']


X_train_transformed = preprocess.fit_transform(X_train)
X_test_transformed = preprocess.transform(X_test)

   HenhouseId        Date  AvgPixelCount  AvgBboxWidth  AvgBboxHeight  \
0         153  2024-05-23     251.546826    155.739795     157.261169   
1         153  2024-05-24     287.958928    165.706255     169.429606   
2         153  2024-05-25     318.004964    174.462789     179.525922   
3         153  2024-05-26     349.519252    185.071027     187.459743   
4         153  2024-05-27     393.534412    197.503473     197.920834   
5         153  2024-05-28     428.508134    207.025527     206.094734   
6         153  2024-05-29     462.056831    215.546985     214.037009   
7         153  2024-05-30     509.581315    228.427108     222.527150   
8         153  2024-05-31     553.637971    238.497947     232.244736   
9         153  2024-06-01     585.624469    245.377165     239.394344   

   NumberOfDaysFromStart  PreviousWeight  AverageWeight  
0                      0            37.0           42.0  
1                      1            42.0           57.0  
2                     

In [15]:
import mlflow
from mlflow.models import infer_signature

In [19]:
import subprocess

subprocess.Popen(['mlflow','ui','--host','0.0.0.0','--port','5000'])


<Popen: returncode: None args: ['mlflow', 'ui', '--host', '0.0.0.0', '--port...>

In [17]:
models_and_params = [
    (LinearRegression(), {
        'fit_intercept': [True, False]
    }),

   
    (Ridge(), {
        'alpha': [0.1, 1.0, 10.0],
        'solver': ['auto', 'svd', 'cholesky', 'lsqr'],
        'fit_intercept': [True, False]
    }),

   
    (Lasso(), {
        'alpha': [0.1, 1.0, 10.0],
        'max_iter': [1000, 2000, 3000],
        'fit_intercept': [True, False]
    }),

    
    (ElasticNet(), {
        'alpha': [0.1, 1.0, 10.0],
        'l1_ratio': [0.2, 0.5, 0.8],
        'max_iter': [1000, 2000, 3000],
        'fit_intercept': [True, False]
    }),

    
    
    (DecisionTreeRegressor(), {
        'criterion': ['squared_error', 'friedman_mse', 'absolute_error', 'poisson'],
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 5, 10]
    }),

    
    (RandomForestRegressor(), {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 5, 10]
    }),

  

   
    (KNeighborsRegressor(), {
        'n_neighbors': [3, 5, 7, 9],
        'weights': ['uniform', 'distance'],
        'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']
    }),

        
    (AdaBoostRegressor(), {
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.1, 0.05, 0.02],
        'loss': ['linear', 'square', 'exponential']
    }),

    
    (GradientBoostingRegressor(), {
        'learning_rate': [0.1, 0.05, 0.02],
        'n_estimators': [50, 100, 150],
        'max_depth': [3, 5, 7, 10],
        'min_samples_split': [3, 5, 10],
        'min_samples_leaf': [2, 5, 10],
    })
]




In [20]:
for model, parameters in models_and_params:
    print(f"\nTrening modela: {model.__class__.__name__}")

   
    grid_search = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train_transformed, y_train)

    
    print(f"Najbolji parametri za {model.__class__.__name__}: {grid_search.best_params_}")

   
    y_pred = grid_search.predict(X_test_transformed)

   
    mae = mean_absolute_error(y_test, y_pred)
    print(f"MAE na test skupu: {mae}")
    print(grid_search.best_score_)


Trening modela: LinearRegression
Najbolji parametri za LinearRegression: {'fit_intercept': False}
MAE na test skupu: 18.32745399838676
-24.81891650717713

Trening modela: Ridge
Najbolji parametri za Ridge: {'alpha': 0.1, 'fit_intercept': True, 'solver': 'auto'}
MAE na test skupu: 45.12722555455027
-62.13338322133261

Trening modela: Lasso
Najbolji parametri za Lasso: {'alpha': 0.1, 'fit_intercept': False, 'max_iter': 2000}
MAE na test skupu: 25.299516656061847
-47.45418287701008

Trening modela: ElasticNet
Najbolji parametri za ElasticNet: {'alpha': 0.1, 'fit_intercept': False, 'l1_ratio': 0.8, 'max_iter': 1000}
MAE na test skupu: 58.17389284788314
-78.62940260382054

Trening modela: DecisionTreeRegressor
Najbolji parametri za DecisionTreeRegressor: {'criterion': 'absolute_error', 'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2}
MAE na test skupu: 30.57706743147453
-54.64970560158186

Trening modela: RandomForestRegressor
Najbolji parametri za RandomForestRegressor: {'m

In [22]:
exp = mlflow.set_experiment(experiment_name="gradiska_train_test")
for model, parameters in models_and_params:

    model_name = type(model).__name__
    print(f"\nTrening modela: {model.__class__.__name__}")

   
    grid_search = GridSearchCV(estimator=model, param_grid=parameters, n_jobs=-1, scoring='neg_mean_absolute_error')
    grid_search.fit(X_train_transformed, y_train)

    
    print(f"Najbolji parametri za {model.__class__.__name__}: {grid_search.best_params_}")

   
    y_pred = grid_search.predict(X_test_transformed)

   
    mae = mean_absolute_error(y_test, y_pred)
    print(f"MAE na test skupu: {mae}")


    metrics = {
       
        'Mean absolute error': mae,
        
    }
    print('Metrics', metrics)



    with mlflow.start_run(experiment_id=exp.experiment_id):
        # Log the hyperparameters
        mlflow.log_param('model_type', model_name)
        mlflow.log_param('columns_used', ", ".join(training_columns))
        mlflow.log_params(grid_search.best_params_)
        mlflow.log_metrics(metrics)
        mlflow.log_param("preprocessing", [step[0] for step in preprocess.steps])

        tags = {
            "task": "Pilici",
            "model":model_name,
            "preprocessing":[step[0] for step in preprocess.steps],
            "data_used": "Gradiska"
        }
        # Set a tag that we can use to remind ourselves what this run was for
        mlflow.set_tags(tags)

        # Infer the model signature
        signature = infer_signature(X_train_transformed, grid_search.predict(X_train_transformed))

        # Log the model
        model_info = mlflow.sklearn.log_model(
            sk_model=grid_search.best_estimator_,
            artifact_path="flow_modeli",
            signature=signature,
            input_example=X_train_transformed,
            registered_model_name=f"{type(model).__name__}_gradiska_stand_3_min_max",
        )

2025/07/30 21:55:49 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.



Trening modela: LinearRegression
Najbolji parametri za LinearRegression: {'fit_intercept': False}
MAE na test skupu: 18.32745399838676
Metrics {'Mean absolute error': 18.32745399838676}


Registered model 'LinearRegression_gradiska_stand_3_min_max' already exists. Creating a new version of this model...
Created version '2' of model 'LinearRegression_gradiska_stand_3_min_max'.



Trening modela: Ridge
Najbolji parametri za Ridge: {'alpha': 0.1, 'fit_intercept': True, 'solver': 'auto'}
MAE na test skupu: 45.12722555455027
Metrics {'Mean absolute error': 45.12722555455027}


2025/07/30 21:55:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'Ridge_gradiska_stand_3_min_max'.
Created version '1' of model 'Ridge_gradiska_stand_3_min_max'.
2025/07/30 21:55:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

2025/07/30 21:56:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'AdaBoostRegressor_gradiska_stand_3_min_max'.
Created version '1' of model 'AdaBoostRegressor_gradiska_stand_3_min_max'.



Trening modela: GradientBoostingRegressor


2025/07/30 21:57:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Najbolji parametri za GradientBoostingRegressor: {'learning_rate': 0.1, 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 100}
MAE na test skupu: 23.719964476926055
Metrics {'Mean absolute error': 23.719964476926055}


Successfully registered model 'GradientBoostingRegressor_gradiska_stand_3_min_max'.
Created version '1' of model 'GradientBoostingRegressor_gradiska_stand_3_min_max'.


In [111]:
grid_search.best_estimator_.coef_

array([    0.        ,  2410.40079273,  -179.23664919,   958.64191949,
       -9809.32923592,  4311.79233262, 11308.06848991,  -351.69097024,
       -4013.27490909, -2869.1762314 ])

In [2]:
!pip install pandas


   ---------------------------------------- 0.0/11.5 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.5 MB 6.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/11.5 MB 4.5 MB/s eta 0:00:03
   -------- ------------------------------- 2.4/11.5 MB 4.6 MB/s eta 0:00:02
   ---------- ----------------------------- 2.9/11.5 MB 4.4 MB/s eta 0:00:02
   ------------- -------------------------- 3.9/11.5 MB 4.4 MB/s eta 0:00:02
   ------------------- -------------------- 5.5/11.5 MB 4.7 MB/s eta 0:00:02
   ------------------------ --------------- 7.1/11.5 MB 5.1 MB/s eta 0:00:01
   ------------------------------ --------- 8.7/11.5 MB 5.4 MB/s eta 0:00:01
   ----------------------------------- ---- 10.2/11.5 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 11.5/11.5 MB 5.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   ---- ----------------------------------- 1.6/12.6 MB 8.3 MB/s eta 0:00:02
   --------


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install scikit-learn


  Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl.metadata (15 kB)
Using cached scikit_learn-1.6.1-cp313-cp313-win_amd64.whl (11.1 MB)



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
